In [1]:
eventDF = sqlContext.read.format('com.databricks.spark.csv').options(header='true', inferSchema='true').load('/mnt/S3/Data/dataSetEvents.csv')

In [2]:
display(eventDF)

In [3]:
def event_tar(row):
  return row.split()[0]
def event_act(row):
  return ' '.join(row.split()[1:len(row)])

In [4]:
# Event target/action 
from pyspark.sql.types import *
from pyspark.sql.functions import *

udf_myFunction = udf(event_tar, StringType() ) 
d=eventDF.withColumn("event_action", udf_myFunction("event"))

udf_myFunction2 = udf(event_act, StringType() ) 
d=d.withColumn("event_type", udf_myFunction2("event"))

In [5]:
# time stuff
from pyspark.sql.functions import * #year, month, dayofmonth
import time
from datetime import *
def to_date(row):
  return row.date()

def to_time(row):
  return row.time().strftime('%H:%M:%S')

udf_myFunction = udf(to_date, DateType() ) 
d1=d.withColumn("Date", udf_myFunction("timestamp"))

udf_myFunction2 = udf(to_time) 
d2=d1.withColumn("Time",udf_myFunction2('timestamp'))



In [6]:
display(d2)

In [7]:
# Vin prefix stuff
def vin_prefix(row):
  return row[0:8]+'_'+row[9]

udf_myFunction4 = udf(vin_prefix,StringType()) 
d3=d2.withColumn("Vin_prefix",udf_myFunction4('vin'))




In [8]:
display(d3)

In [9]:
sqlContext.registerDataFrameAsTable(d3,'Table1')

In [10]:
YMM_df=sqlContext.sql('SELECT distinct(vin_prefix),year,make,model FROM Table1 ORDER BY make, model, year, vin_prefix')

In [11]:
display(YMM_df)

In [12]:
VIN_df=sqlContext.sql('SELECT distinct(VIN),vin_prefix,trim,body_style,cab_style FROM Table1 ORDER BY VIN')
display(VIN_df)

In [13]:
display(VIN_df)

In [14]:
Location_df=sqlContext.sql('SELECT distinct(string(ZIP)),city,state FROM Table1 ORDER BY ZIP, city, state')



In [15]:
display(Location_df)

In [16]:
Location_df.write.csv('location2.csv')

In [17]:
display(Location_df)

In [18]:
Listing_fact_df=sqlContext.sql('SELECT VIN,date,string(ZIP),condition,price,mileage,image_count FROM Table1 ORDER BY VIN,date')
Listing_fact_df=Listing_fact_df.distinct()


In [19]:
display(Listing_fact_df)

In [20]:
Listing_fact_df=Listing_fact_df.distinct()
display(Listing_fact_df)

In [21]:
event_fact_df=sqlContext.sql('SELECT user_id,VIN,date,ZIP,referring_domain,event_type,event_action,time, COUNT(event_type) as count_event_type FROM Table1 GROUP BY user_id,VIN,date,ZIP,condition,referring_domain,event_action,event_type,time ORDER BY user_id,VIN,date,time,event_action,event_type')
display(event_fact_df)

In [22]:
display(Listing_fact_df)

In [23]:
display(event_fact_df)

In [24]:
sqlContext.registerDataFrameAsTable(event_fact_df,'event_fact_df')
sqlContext.registerDataFrameAsTable(Listing_fact_df,'Listing_fact_df')
sqlContext.registerDataFrameAsTable(VIN_df,'VIN_df')
sqlContext.registerDataFrameAsTable(YMM_df,'YMM_df')
sqlContext.registerDataFrameAsTable(Location_df,'Location_df')

In [25]:
test40=sqlContext.sql('SELECT event_fact_df.user_id,Listing_fact_df.condition,Listing_fact_df.price,Listing_fact_df.mileage,event_fact_df.VIN, event_fact_df.date, event_fact_df.count_event_type FROM Listing_fact_df,event_fact_df WHERE Listing_fact_df.VIN=event_fact_df.VIN AND Listing_fact_df.date=event_fact_df.date')
display(test40)


In [27]:
sqlContext.registerDataFrameAsTable(test40,'test40')

In [28]:
test41=sqlContext.sql('SELECT user_id,condition,price,mileage,test40.VIN,date,COUNT(count_event_type) as event_count,YMM_df.year FROM test40,YMM_df,VIN_df WHERE VIN_df.vin_prefix=YMM_df.vin_prefix AND VIN_df.VIN=test40.VIN GROUP BY user_id,condition,price,mileage,test40.VIN,date,YMM_df.year')
display(test41)

In [29]:
sqlContext.registerDataFrameAsTable(test41,'test41')


In [30]:
test41=sqlContext.sql('SELECT user_id,condition,price,mileage,VIN,SUM(event_count),year FROM test41 GROUP BY user_id,condition,price,mileage,VIN,year ORDER BY user_id')


In [31]:
display(test41)

In [33]:
# Distincts the Listing
dis_list=sqlContext.sql('SELECT distinct(VIN),Listing_fact_df.condition, Listing_fact_df.price, Listing_fact_df.mileage FROM Listing_fact_df')
display(dis_list)

In [34]:
# distinct VIN from listing
sqlContext.registerDataFrameAsTable(dis_list,'dis_list_df')

In [35]:
test2=sqlContext.sql('SELECT event_fact_df.user_id, event_fact_df.VIN,COUNT(event_fact_df.count_event_type),dis_list_df.condition, dis_list_df.price, dis_list_df.mileage,YMM_df.year FROM event_fact_df,dis_list_df,YMM_df,VIN_df WHERE event_fact_df.VIN=dis_list_df.VIN AND VIN_df.VIN=event_fact_df.VIN AND VIN_df.vin_prefix=YMM_df.vin_prefix AND VIN_df.VIN=dis_list_df.VIN GROUP BY event_fact_df.user_id,event_fact_df.VIN,dis_list_df.condition,dis_list_df.price,dis_list_df.mileage,YMM_df.year ORDER BY user_id')
display(test2)

In [36]:
test2.write.csv('Model.csv')

In [37]:
# OG sql code
#test2=sqlContext.sql('SELECT event_fact_df.user_id, event_fact_df.VIN,COUNT(event_fact_df.count_event_type),Listing_fact_df.condition, Listing_fact_df.price, Listing_fact_df.mileage,YMM_df.year FROM event_fact_df,Listing_fact_df,YMM_df,VIN_df WHERE event_fact_df.VIN=Listing_fact_df.VIN AND VIN_df.VIN=event_fact_df.VIN AND VIN_df.vin_prefix=YMM_df.vin_prefix AND VIN_df.VIN=Listing_fact_df.VIN GROUP BY event_fact_df.user_id,event_fact_df.VIN,Listing_fact_df.condition,Listing_fact_df.price,Listing_fact_df.mileage,YMM_df.year ORDER BY user_id')

#display(test2)

In [38]:
#test3=sqlContext.sql('SELECT event_fact_df.user_id, event_fact_df.VIN,Listing_fact_df.condition, Listing_fact_df.price, Listing_fact_df.mileage,YMM_df.year FROM event_fact_df,Listing_fact_df,YMM_df,VIN_df WHERE event_fact_df.VIN=Listing_fact_df.VIN AND VIN_df.VIN=event_fact_df.VIN AND VIN_df.vin_prefix=YMM_df.vin_prefix ORDER BY user_id')

#display(test3)